## Graph subsampling

In [16]:
%reset -f

In [17]:
%pip install ..

Processing /home/sean/Workspaces/contact-networks
  Preparing metadata (setup.py) ... done
Using legacy 'setup.py install' for contact-networks, since package 'wheel' is not installed.
  Attempting uninstall: contact-networks
    Found existing installation: contact-networks 0.1.0
    Uninstalling contact-networks-0.1.0:
      Successfully uninstalled contact-networks-0.1.0
  Running setup.py install for contact-networks ... done
Note: you may need to restart the kernel to use updated packages.


In [18]:
import os
import numpy as np
import matplotlib.pyplot as plt
from tabulate import tabulate
from IPython.display import Markdown
from contact_networks import graph_metrics, sampling_methods
from contact_networks import utils

In [19]:
np.seterr(divide='raise')

{'divide': 'raise', 'over': 'warn', 'under': 'ignore', 'invalid': 'warn'}

In [20]:
data = utils.load_data()

In [21]:
FIG_DIR="../figures/"
TBL_DIR="../tables/"

### Graphing helper-functions

In [22]:
def _hist(ax, xs):
    # histogram
    hist, bins = np.histogram(xs, bins=20)
    bar = ax.bar(bins[:-1], hist, 
                 width=np.diff(bins), 
                 edgecolor='black', 
                 linewidth=0.5,
                 alpha=0.7)
    
    # hide all except the bottom spine
    ax.spines["right"].set_visible(False)
    ax.spines["left"].set_visible(False)
    ax.spines["top"].set_visible(False)
    
    return bar

In [23]:
def _subfig(axes, item, metric):
    axes = axes.ravel()

    _hist(
        axes[0], 
        metric(item['face-to-face'])
    )
    axes[0].set_title(f"Face-to-face network", fontsize=10)

    _hist(
        axes[1], 
        metric(item['co-present'])
    )
    axes[1].set_title(f"Co-presence network", fontsize=10)

In [24]:
def _supfig(data, title, metric=graph_metrics.degree_distribution):
    fig = plt.figure(layout='constrained', figsize=(8,11))
    subfigs = fig.subfigures(6,1, hspace=0.05)

    for subfig, key in zip(subfigs.ravel(), data):
        axes = subfig.subplots(1,2, sharey=True)
        _subfig(axes, data[key], metric)
        subfig.suptitle(f"{key} dataset, |V|={data[key]['face-to-face'].shape[0]}", 
                        fontsize=10, fontweight='bold')

    fig.suptitle(title, fontsize=12, fontweight='bold')
    return fig


### Table helper-functions

In [25]:
def compute_graph_metric(data, graph_metric, sampling_method, 
                         sized_by_nodes, sampling_ratios):
    metrics = np.zeros((6, 2 * sampling_ratios.size + 3), dtype=object)

    n_ratios = sampling_ratios.size
    row_idx = 0
    for key, item in data.items():
        adj_f2f = item['face-to-face']
        adj_cop = item['co-present']

        metrics[row_idx, 0] = key
        metrics[row_idx, 1] = graph_metric(adj_f2f)
        metrics[row_idx, n_ratios + 2] = graph_metric(adj_cop)

        if sized_by_nodes:
            size_f2f = adj_f2f.shape[0]
            size_cop = adj_cop.shape[0]
        else:
            size_f2f = np.count_nonzero(adj_f2f) / 2
            size_cop = np.count_nonzero(adj_cop) / 2

        for col_idx, ratio in enumerate(sampling_ratios):
            # compute adjacency matrices of sub-graphs
            sub_f2f = sampling_method(adj_f2f, int(ratio * size_f2f))
            sub_cop = sampling_method(adj_cop, int(ratio * size_cop))

            # remove disconnected nodes
            mask = np.any(sub_f2f != 0, axis=0)
            sub_f2f = sub_f2f[mask][:, mask]

            mask = np.any(sub_cop != 0, axis=0)
            sub_cop = sub_cop[mask][:, mask]

            # compute/store graph metric
            metrics[row_idx, col_idx + 2] = graph_metric(sub_f2f)
            metrics[row_idx, col_idx + n_ratios + 3] = graph_metric(sub_cop)

        row_idx += 1

    return metrics

In [26]:
def tabulate_graph_metric(data, graph_metric, sampling_method, 
                          sized_by_nodes,sampling_ratios, 
                          filename, floatfmt, title):
    metrics = compute_graph_metric(data, graph_metric, sampling_method, 
                                   sized_by_nodes, sampling_ratios)

    headers = ["Dataset", "Face-to-Face"] \
            + [f"f={ratio}" for ratio in sampling_ratios] \
            + ["Co-present"] \
            + [f"f={ratio}" for ratio in sampling_ratios]

    # create and save a LaTeX table
    tex_title = r"\caption{"+title+"}"
    tex_begin_table = r"\begin{table}[ht]"
    tex_end_table = r"\end{table}"

    tex_table = tabulate(
        metrics.tolist(),
        headers=headers,
        floatfmt=floatfmt,
        tablefmt="latex"  # Set the table format to LaTeX
    )

    tex_table = f"{tex_begin_table}\n{tex_table}\n{tex_title}\n{tex_end_table}"

    with open(os.path.join(TBL_DIR,filename+".tex"), "w") as f:
        f.write(tex_table)

    # create and display a Markodown table
    md_title = title+"\n"

    md_table = tabulate(
        metrics.tolist(),
        headers=headers,
        floatfmt=floatfmt,
        tablefmt="github"  # Set the table format to LaTeX
    )

    md_table = f"{md_title}\n{md_table}"

    display(Markdown(md_table))

In [27]:
sampling_ratios = np.array([0.9, 0.8, 0.7, 0.6, 0.5])

### Density

In [28]:
floatfmt = ("", ".3f", ".3f", ".3f", ".3f", ".3f", ".3f", ".3f", ".3f", ".3f", ".3f", ".3f", ".3f")

In [29]:
gms = [
    {
        'gm': graph_metrics.density,
        'name': "graph_metrics.density",
        'floatfmt': ()
    },
    {
        'gm': graph_metrics.average_degree,
        'name': "graph_metrics.average_degree",
        'floatfmt': (),
    },
    {
        'gm': graph_metrics.average_clustering_coefficient,
        'name': "graph_metrics.average_clustering_coefficient",
        'floatfmt': ()
    }
]
sms = [
    {
        'sm': sampling_methods.node_sampling,
        'name': "sampling_methods.node_sampling",
        'sized_by_nodes': True
    },
    {
        'sm': sampling_methods.edge_sampling,
        'name': "sampling_methods.edge_sampling",
        'sized_by_nodes': False
    },
    {
        'sm': sampling_methods.metropolis_hastings_rw,
        'name': "sampling_methods.metropolis_hastings_rw",
        'sized_by_nodes': False
    },
    {
        'sm': sampling_methods.frontier_sampling,
        'name': "sampling_methods.frontier_sampling",
        'sized_by_nodes': False
    },
    {
        'sm': sampling_methods.snowball_expansion_sampling,
        'name': "sampling_methods.snowball_expansion_sampling",
        'sized_by_nodes': True
    }
]

In [30]:
idx = 1
for gm in gms:
    for sm in sms:
        tabulate_graph_metric(data=data, 
            graph_metric=gm['gm'],
            sampling_method=sm['sm'],
            sized_by_nodes=sm['sized_by_nodes'],
            sampling_ratios=sampling_ratios, 
            filename=f"proj2_tab{idx}",
            floatfmt=floatfmt,
            title=f"`{gm['name']}` using `{sm['name']}`")
        idx += 1

`graph_metrics.density` using `sampling_methods.node_sampling`

| Dataset    |   Face-to-Face |   f=0.9 |   f=0.8 |   f=0.7 |   f=0.6 |   f=0.5 |   Co-present |   f=0.9 |   f=0.8 |   f=0.7 |   f=0.6 |   f=0.5 |
|------------|----------------|---------|---------|---------|---------|---------|--------------|---------|---------|---------|---------|---------|
| Thiers13   |          0.109 |   0.106 |   0.112 |   0.110 |   0.109 |   0.103 |        0.811 |   0.808 |   0.810 |   0.813 |   0.806 |   0.782 |
| InVS13     |          0.180 |   0.182 |   0.177 |   0.183 |   0.183 |   0.145 |        0.877 |   0.872 |   0.889 |   0.887 |   0.882 |   0.893 |
| LyonSchool |          0.285 |   0.287 |   0.277 |   0.284 |   0.292 |   0.291 |        0.912 |   0.918 |   0.908 |   0.924 |   0.922 |   0.917 |
| SFHH       |          0.118 |   0.118 |   0.116 |   0.122 |   0.110 |   0.111 |        0.908 |   0.903 |   0.905 |   0.904 |   0.904 |   0.913 |
| LH10       |          0.406 |   0.399 |   0.398 |   0.324 |   0.501 |   0.407 |        0.525 |   0.493 |   0.483 |   0.500 |   0.532 |   0.544 |
| InVS15     |          0.182 |   0.182 |   0.182 |   0.181 |   0.185 |   0.173 |        0.701 |   0.706 |   0.715 |   0.687 |   0.701 |   0.678 |

`graph_metrics.density` using `sampling_methods.edge_sampling`

| Dataset    |   Face-to-Face |   f=0.9 |   f=0.8 |   f=0.7 |   f=0.6 |   f=0.5 |   Co-present |   f=0.9 |   f=0.8 |   f=0.7 |   f=0.6 |   f=0.5 |
|------------|----------------|---------|---------|---------|---------|---------|--------------|---------|---------|---------|---------|---------|
| Thiers13   |          0.109 |   0.098 |   0.087 |   0.076 |   0.065 |   0.055 |        0.811 |   0.730 |   0.649 |   0.568 |   0.487 |   0.406 |
| InVS13     |          0.180 |   0.162 |   0.144 |   0.126 |   0.108 |   0.090 |        0.877 |   0.789 |   0.701 |   0.614 |   0.526 |   0.438 |
| LyonSchool |          0.285 |   0.257 |   0.228 |   0.200 |   0.171 |   0.143 |        0.912 |   0.821 |   0.730 |   0.638 |   0.547 |   0.456 |
| SFHH       |          0.118 |   0.106 |   0.095 |   0.083 |   0.071 |   0.060 |        0.908 |   0.817 |   0.726 |   0.636 |   0.545 |   0.454 |
| LH10       |          0.406 |   0.365 |   0.324 |   0.284 |   0.243 |   0.203 |        0.525 |   0.473 |   0.420 |   0.368 |   0.315 |   0.263 |
| InVS15     |          0.182 |   0.166 |   0.146 |   0.129 |   0.109 |   0.092 |        0.701 |   0.631 |   0.561 |   0.495 |   0.420 |   0.354 |

`graph_metrics.density` using `sampling_methods.metropolis_hastings_rw`

| Dataset    |   Face-to-Face |   f=0.9 |   f=0.8 |   f=0.7 |   f=0.6 |   f=0.5 |   Co-present |   f=0.9 |   f=0.8 |   f=0.7 |   f=0.6 |   f=0.5 |
|------------|----------------|---------|---------|---------|---------|---------|--------------|---------|---------|---------|---------|---------|
| Thiers13   |          0.109 |   0.062 |   0.058 |   0.053 |   0.047 |   0.042 |        0.811 |   0.478 |   0.445 |   0.405 |   0.364 |   0.317 |
| InVS13     |          0.180 |   0.101 |   0.096 |   0.094 |   0.088 |   0.068 |        0.877 |   0.515 |   0.484 |   0.439 |   0.397 |   0.342 |
| LyonSchool |          0.285 |   0.162 |   0.151 |   0.138 |   0.126 |   0.108 |        0.912 |   0.539 |   0.503 |   0.458 |   0.411 |   0.358 |
| SFHH       |          0.118 |   0.067 |   0.062 |   0.058 |   0.050 |   0.045 |        0.908 |   0.537 |   0.501 |   0.454 |   0.408 |   0.357 |
| LH10       |          0.406 |   0.238 |   0.216 |   0.192 |   0.182 |   0.156 |        0.525 |   0.311 |   0.295 |   0.273 |   0.243 |   0.199 |
| InVS15     |          0.182 |   0.108 |   0.098 |   0.092 |   0.083 |   0.070 |        0.701 |   0.410 |   0.391 |   0.349 |   0.317 |   0.271 |

`graph_metrics.density` using `sampling_methods.frontier_sampling`

| Dataset    |   Face-to-Face |   f=0.9 |   f=0.8 |   f=0.7 |   f=0.6 |   f=0.5 |   Co-present |   f=0.9 |   f=0.8 |   f=0.7 |   f=0.6 |   f=0.5 |
|------------|----------------|---------|---------|---------|---------|---------|--------------|---------|---------|---------|---------|---------|
| Thiers13   |          0.109 |   0.064 |   0.060 |   0.054 |   0.048 |   0.043 |        0.811 |   0.480 |   0.445 |   0.407 |   0.364 |   0.318 |
| InVS13     |          0.180 |   0.107 |   0.099 |   0.091 |   0.083 |   0.072 |        0.877 |   0.522 |   0.480 |   0.445 |   0.397 |   0.342 |
| LyonSchool |          0.285 |   0.169 |   0.156 |   0.142 |   0.126 |   0.111 |        0.912 |   0.539 |   0.501 |   0.457 |   0.411 |   0.359 |
| SFHH       |          0.118 |   0.070 |   0.065 |   0.060 |   0.053 |   0.046 |        0.908 |   0.537 |   0.498 |   0.456 |   0.409 |   0.358 |
| LH10       |          0.406 |   0.241 |   0.220 |   0.197 |   0.178 |   0.161 |        0.525 |   0.319 |   0.287 |   0.269 |   0.237 |   0.209 |
| InVS15     |          0.182 |   0.110 |   0.102 |   0.092 |   0.083 |   0.071 |        0.701 |   0.417 |   0.387 |   0.351 |   0.317 |   0.278 |

`graph_metrics.density` using `sampling_methods.snowball_expansion_sampling`

| Dataset    |   Face-to-Face |   f=0.9 |   f=0.8 |   f=0.7 |   f=0.6 |   f=0.5 |   Co-present |   f=0.9 |   f=0.8 |   f=0.7 |   f=0.6 |   f=0.5 |
|------------|----------------|---------|---------|---------|---------|---------|--------------|---------|---------|---------|---------|---------|
| Thiers13   |          0.109 |   0.112 |   0.120 |   0.124 |   0.131 |   0.142 |        0.811 |   0.806 |   0.805 |   0.807 |   0.813 |   0.816 |
| InVS13     |          0.180 |   0.191 |   0.195 |   0.212 |   0.222 |   0.212 |        0.877 |   0.866 |   0.893 |   0.899 |   0.885 |   0.871 |
| LyonSchool |          0.285 |   0.289 |   0.292 |   0.306 |   0.325 |   0.360 |        0.912 |   0.914 |   0.922 |   0.924 |   0.911 |   0.930 |
| SFHH       |          0.118 |   0.120 |   0.122 |   0.134 |   0.133 |   0.145 |        0.908 |   0.910 |   0.908 |   0.926 |   0.936 |   0.946 |
| LH10       |          0.406 |   0.437 |   0.495 |   0.504 |   0.527 |   0.587 |        0.525 |   0.561 |   0.556 |   0.660 |   0.717 |   0.811 |
| InVS15     |          0.182 |   0.179 |   0.187 |   0.186 |   0.198 |   0.200 |        0.701 |   0.699 |   0.693 |   0.690 |   0.700 |   0.709 |

...

`graph_metrics.average_clustering_coefficient` using `sampling_methods.node_sampling`

| Dataset    |   Face-to-Face |   f=0.9 |   f=0.8 |   f=0.7 |   f=0.6 |   f=0.5 |   Co-present |   f=0.9 |   f=0.8 |   f=0.7 |   f=0.6 |   f=0.5 |
|------------|----------------|---------|---------|---------|---------|---------|--------------|---------|---------|---------|---------|---------|
| Thiers13   |          0.504 |   0.502 |   0.506 |   0.510 |   0.495 |   0.515 |        0.843 |   0.840 |   0.840 |   0.843 |   0.828 |   0.837 |
| InVS13     |          0.426 |   0.423 |   0.417 |   0.448 |   0.413 |   0.420 |        0.928 |   0.929 |   0.928 |   0.937 |   0.924 |   0.927 |
| LyonSchool |          0.526 |   0.533 |   0.537 |   0.522 |   0.525 |   0.553 |        0.929 |   0.927 |   0.927 |   0.933 |   0.929 |   0.926 |
| SFHH       |          0.282 |   0.283 |   0.293 |   0.254 |   0.253 |   0.310 |        0.944 |   0.942 |   0.941 |   0.948 |   0.944 |   0.949 |
| LH10       |          0.679 |   0.692 |   0.652 |   0.705 |   0.687 |   0.660 |        0.776 |   0.764 |   0.788 |   0.809 |   0.825 |   0.730 |
| InVS15     |          0.381 |   0.384 |   0.391 |   0.382 |   0.370 |   0.391 |        0.799 |   0.799 |   0.804 |   0.791 |   0.810 |   0.832 |

`graph_metrics.average_clustering_coefficient` using `sampling_methods.edge_sampling`

| Dataset    |   Face-to-Face |   f=0.9 |   f=0.8 |   f=0.7 |   f=0.6 |   f=0.5 |   Co-present |   f=0.9 |   f=0.8 |   f=0.7 |   f=0.6 |   f=0.5 |
|------------|----------------|---------|---------|---------|---------|---------|--------------|---------|---------|---------|---------|---------|
| Thiers13   |          0.504 |   0.453 |   0.412 |   0.349 |   0.304 |   0.251 |        0.843 |   0.758 |   0.674 |   0.590 |   0.505 |   0.422 |
| InVS13     |          0.426 |   0.384 |   0.333 |   0.298 |   0.260 |   0.218 |        0.928 |   0.836 |   0.737 |   0.652 |   0.553 |   0.463 |
| LyonSchool |          0.526 |   0.471 |   0.425 |   0.365 |   0.314 |   0.263 |        0.929 |   0.837 |   0.742 |   0.650 |   0.556 |   0.463 |
| SFHH       |          0.282 |   0.254 |   0.224 |   0.193 |   0.168 |   0.144 |        0.944 |   0.849 |   0.756 |   0.660 |   0.567 |   0.472 |
| LH10       |          0.679 |   0.605 |   0.546 |   0.479 |   0.427 |   0.307 |        0.776 |   0.698 |   0.610 |   0.536 |   0.450 |   0.360 |
| InVS15     |          0.381 |   0.344 |   0.308 |   0.277 |   0.232 |   0.189 |        0.799 |   0.719 |   0.639 |   0.559 |   0.484 |   0.399 |

`graph_metrics.average_clustering_coefficient` using `sampling_methods.metropolis_hastings_rw`

| Dataset    |   Face-to-Face |   f=0.9 |   f=0.8 |   f=0.7 |   f=0.6 |   f=0.5 |   Co-present |   f=0.9 |   f=0.8 |   f=0.7 |   f=0.6 |   f=0.5 |
|------------|----------------|---------|---------|---------|---------|---------|--------------|---------|---------|---------|---------|---------|
| Thiers13   |          0.504 |   0.288 |   0.276 |   0.246 |   0.232 |   0.197 |        0.843 |   0.497 |   0.461 |   0.420 |   0.381 |   0.330 |
| InVS13     |          0.426 |   0.239 |   0.238 |   0.186 |   0.230 |   0.156 |        0.928 |   0.544 |   0.512 |   0.464 |   0.421 |   0.362 |
| LyonSchool |          0.526 |   0.300 |   0.283 |   0.261 |   0.230 |   0.209 |        0.929 |   0.552 |   0.514 |   0.466 |   0.418 |   0.366 |
| SFHH       |          0.282 |   0.133 |   0.128 |   0.112 |   0.103 |   0.084 |        0.944 |   0.562 |   0.518 |   0.475 |   0.425 |   0.371 |
| LH10       |          0.679 |   0.333 |   0.314 |   0.312 |   0.256 |   0.208 |        0.776 |   0.415 |   0.401 |   0.389 |   0.343 |   0.315 |
| InVS15     |          0.381 |   0.224 |   0.197 |   0.184 |   0.162 |   0.156 |        0.799 |   0.462 |   0.433 |   0.397 |   0.357 |   0.308 |

`graph_metrics.average_clustering_coefficient` using `sampling_methods.frontier_sampling`

| Dataset    |   Face-to-Face |   f=0.9 |   f=0.8 |   f=0.7 |   f=0.6 |   f=0.5 |   Co-present |   f=0.9 |   f=0.8 |   f=0.7 |   f=0.6 |   f=0.5 |
|------------|----------------|---------|---------|---------|---------|---------|--------------|---------|---------|---------|---------|---------|
| Thiers13   |          0.504 |   0.304 |   0.267 |   0.261 |   0.236 |   0.212 |        0.843 |   0.499 |   0.463 |   0.427 |   0.382 |   0.334 |
| InVS13     |          0.426 |   0.248 |   0.247 |   0.187 |   0.245 |   0.186 |        0.928 |   0.554 |   0.517 |   0.468 |   0.425 |   0.367 |
| LyonSchool |          0.526 |   0.313 |   0.286 |   0.260 |   0.241 |   0.209 |        0.929 |   0.552 |   0.515 |   0.469 |   0.422 |   0.372 |
| SFHH       |          0.282 |   0.173 |   0.160 |   0.135 |   0.129 |   0.114 |        0.944 |   0.562 |   0.520 |   0.475 |   0.428 |   0.371 |
| LH10       |          0.679 |   0.432 |   0.363 |   0.321 |   0.295 |   0.267 |        0.776 |   0.450 |   0.444 |   0.419 |   0.348 |   0.285 |
| InVS15     |          0.381 |   0.231 |   0.213 |   0.201 |   0.170 |   0.169 |        0.799 |   0.469 |   0.441 |   0.406 |   0.360 |   0.319 |

`graph_metrics.average_clustering_coefficient` using `sampling_methods.snowball_expansion_sampling`

| Dataset    |   Face-to-Face |   f=0.9 |   f=0.8 |   f=0.7 |   f=0.6 |   f=0.5 |   Co-present |   f=0.9 |   f=0.8 |   f=0.7 |   f=0.6 |   f=0.5 |
|------------|----------------|---------|---------|---------|---------|---------|--------------|---------|---------|---------|---------|---------|
| Thiers13   |          0.504 |   0.515 |   0.529 |   0.519 |   0.531 |   0.542 |        0.843 |   0.839 |   0.841 |   0.843 |   0.850 |   0.858 |
| InVS13     |          0.426 |   0.424 |   0.426 |   0.408 |   0.385 |   0.411 |        0.928 |   0.921 |   0.937 |   0.949 |   0.942 |   0.936 |
| LyonSchool |          0.526 |   0.525 |   0.525 |   0.537 |   0.558 |   0.575 |        0.929 |   0.931 |   0.938 |   0.940 |   0.929 |   0.945 |
| SFHH       |          0.282 |   0.289 |   0.301 |   0.311 |   0.312 |   0.328 |        0.944 |   0.946 |   0.946 |   0.952 |   0.956 |   0.964 |
| LH10       |          0.679 |   0.684 |   0.705 |   0.730 |   0.745 |   0.778 |        0.776 |   0.777 |   0.789 |   0.815 |   0.841 |   0.904 |
| InVS15     |          0.381 |   0.380 |   0.384 |   0.390 |   0.401 |   0.414 |        0.799 |   0.801 |   0.792 |   0.789 |   0.788 |   0.803 |